<a href="https://colab.research.google.com/github/R-aryan/Name_Entity_Recognition/blob/master/NER_DiseaeName_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
print("Hello colab")

Hello colab


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install -r '/content/drive/My Drive/Colab Notebooks/requirements.txt'

     |████████████████████████████████| 2.8MB 7.9MB/s 
     |████████████████████████████████| 3.7MB 44.5MB/s 
     |████████████████████████████████| 133kB 69.8MB/s 
     |████████████████████████████████| 5.9MB 41.3MB/s 
ERROR: Could not find a version that satisfies the requirement en-ner-bc5cdr-md==0.2.4 (from -r /content/drive/My Drive/Colab Notebooks/requirements.txt (line 21)) (from versions: none)
ERROR: No matching distribution found for en-ner-bc5cdr-md==0.2.4 (from -r /content/drive/My Drive/Colab Notebooks/requirements.txt (line 21))


In [0]:
pip install pytorch-pretrained-bert==0.4.0

In [0]:
pip install scispacy

In [0]:
pip install '/content/drive/My Drive/Colab Notebooks/en_ner_bc5cdr_md-0.2.4.tar.gz'

Processing ./drive/My Drive/Colab Notebooks/en_ner_bc5cdr_md-0.2.4.tar.gz
  Created wheel for en-ner-bc5cdr-md: filename=en_ner_bc5cdr_md-0.2.4-cp36-none-any.whl size=70531466 sha256=857ac063413e467852cf324a9b984a9d3d18802767279a0208c74e8a91b9962b
  Stored in directory: /root/.cache/pip/wheels/5f/07/ca/bda9e50b07b11aa9a6e51441a9dda435560fc548957a31b81e
Successfully built en-ner-bc5cdr-md


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")




import tensorflow_hub as hub
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense
from keras.layers import TimeDistributed, Dropout, Bidirectional

from tqdm import tqdm, trange

In [0]:
# Defining Constants
 
# Maximum length of text sentences
MAXLEN = 180

# batch size
BS=48

In [0]:
data_test = pd.read_csv("/content/drive/My Drive/NER_Disease/train.csv", encoding="latin1")
data1 = data_test.sample(n=15000,random_state=2020)
data1.tail(10)

# data1=data.sample(n=25000,random_state=2020)
# data1.head(10)

,id,Doc_ID,Sent_ID,Word,tag
2059180,2059181,13684,87015,adhesion,O
4386570,4386571,28960,184768,<,O
3864848,3864849,25525,162691,the,O
4106376,4106377,27126,172912,5-HT,O
1403106,1403107,9314,59117,in,O
3287894,3287895,21719,138403,in,O
4195244,4195245,27724,176684,the,O
2475003,2475004,16324,104084,follow,O
1521318,1521319,10130,64195,the,O
2499807,2499808,16493,105121,care,O


In [0]:
words = list(set(data1["Word"].values))
words.append("ENDPAD")
n_words = len(words); n_words
#print(words[1:10])

5260

In [0]:
tags = list(set(data1["tag"].values))
n_tags = len(tags); n_tags
print(tags[0:n_tags])

['O', 'I-indications', 'B-indications']


In [0]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                                           s["tag"].values.tolist())]
        self.grouped = self.data.groupby("Sent_ID").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [0]:
getter = SentenceGetter(data1)

In [0]:
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'SW620'

In [0]:
labels = [[s[1] for s in sent] for sent in getter.sentences]
print(len(labels))

14247


In [0]:
tags_vals = list(set(data1["tag"].values))
tag2idx = {t: i for i, t in enumerate(tags_vals)}

print(tag2idx)

{'O': 0, 'I-indications': 1, 'B-indications': 2}


In [0]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
device

device(type='cuda')

The Bert implementation comes with a pretrained tokenizer and a definied vocabulary. We load the one related to the smallest pre-trained model bert-base-uncased.  cased variate since  is well suited for NER.

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Now we tokenize all sentences

In [0]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print(tokenized_texts[0])

['sw', '##6', '##20']


Next, we cut and pad the token and label sequences to our desired length. That is Next, we need to convert each token in each review to an id as present in the tokenizer vocabulary. If there’s a token that is not present in the vocabulary, the tokenizer will use the special [UNK] token and use its id:

Refer:-  https://towardsdatascience.com/bert-to-the-rescue-17671379687f

In [0]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAXLEN, dtype="long", truncating="post", padding="post")
input_ids[0]

array([25430,  2575, 11387,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0])

In [0]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAXLEN, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")

len(tags)

14247

The Bert model supports something called attention_mask, which is similar to the masking in keras. So here we create the mask to ignore the padded elements in the sequences.

In [0]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

#attention_masks

In [0]:
data1["tag"].values

array(['O', 'O', 'O', ..., 'O', 'O', 'O'], dtype=object)

Now we split the dataset to use 10% to validate the model.

In [0]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018,test_size=0.05)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018,test_size=0.05)

Since we’re operating in pytorch, we have to convert the dataset to torch tensors.

In [0]:
tr_inputs = torch.tensor(tr_inputs).to(torch.int64)
val_inputs = torch.tensor(val_inputs).to(torch.int64)
tr_tags = torch.tensor(tr_tags).to(torch.int64)
val_tags = torch.tensor(val_tags).to(torch.int64)
tr_masks = torch.tensor(tr_masks).to(torch.int64)
val_masks = torch.tensor(val_masks).to(torch.int64)

The last step is to define the dataloaders. We shuffle the data at training time with the RandomSampler and at test time we just pass them sequentially with the SequentialSampler.

In [0]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

## Setup the Bert model for finetuning

The pytorch-pretrained-bert package provides a BertForTokenClassification class for token-level predictions. BertForTokenClassification is a fine-tuning model that wraps BertModel and adds token-level classifier on top of the BertModel. The token-level classifier is a linear layer that takes as input the last hidden state of the sequence. We load the pre-trained bert-base-uncased model and provide the number of possible labels.

In [0]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx))

In [0]:
# print("Model's state_dict:")
# for param_tensor in model.state_dict():
#     print(param_tensor, "\t", model.state_dict()[param_tensor].size())

import os



try:
  corpus = "/content/drive/My Drive/Colab Notebooks/bert_epoch_state.pt"
  checkpoint = torch.load(corpus)
  model.load_state_dict(checkpoint['model_state_dict'])
  epoch = checkpoint['epoch']
  loss = checkpoint['loss']



except:
  pass


/content/drive/My Drive/Colab Notebooks/bert_epoch_state.pt


Now we have to pass the model parameters to the CPU.

In [0]:
model.cpu()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
       

Before we can start the fine-tuning process, we have to setup the optimizer and add the parameters it should update. A common choice is the Adam optimizer. We also add some weight_decay as regularization to the main weight matrices. If you have limited resources, you can also try to just train the linear classifier on top of Bert and keep all other weights fixed. This will still give you a good performance.

In [0]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    #print(param_optimizer)
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

## Finetune Bert

First we define some metrics, we want to track while training. We use the f1_score from the seqeval package.And we use simple accuracy on a token level comparable to the accuracy in keras.

In [0]:
pip install seqeval

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=75dcc99864bdc5231b6e846311cc3ea63cdd6e81ceb1a654530ba7c6af6fbf4e
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [0]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
model.cuda(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
       

Finally, we can fine-tune the model. A few epochs should be enough. The paper suggest 3-4 epochs.

In [0]:
epochs = 5
max_grad_norm = 1.0

model.cuda(device)

for epoch in trange(epochs, desc="Epoch"):

    # TRAIN loop
    model.train()

    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):

        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        
        # backward pass
        loss.backward()

        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)

        # update parameters
        optimizer.step()
        model.zero_grad()

    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))



    # VALIDATION on validation set
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    predictions , true_labels = [], []

    for batch in valid_dataloader:

        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():

            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
            
        logits = logits.detach().cpu().numpy()

        label_ids = b_labels.to('cpu').numpy()

        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])

        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1

    eval_loss = eval_loss/nb_eval_steps

    print("Validation loss: {}".format(eval_loss))

    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]

    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))


   






torch.save(model.state_dict(), '/content/drive/My Drive/NER_Disease/my_NER_model.h5')
    


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Train loss: 0.010535766553129874
Validation loss: 0.003995094417326111
Validation Accuracy: 0.9995652173913046
F1-Score: 0



Epoch:  20%|██        | 1/5 [03:41<14:44, 221.19s/it]

Train loss: 0.0026672742769948314
Validation loss: 0.003629560764871396
Validation Accuracy: 0.9995652173913046
F1-Score: 0



Epoch:  40%|████      | 2/5 [07:22<11:03, 221.32s/it]

...


Epoch:  60%|██████    | 3/5 [11:04<07:22, 221.36s/it]

Train loss: 0.0018797821331617585
Validation loss: 0.0018063113915875715
Validation Accuracy: 0.9995652173913046
F1-Score: 0



Epoch:  80%|████████  | 4/5 [14:45<03:41, 221.38s/it]

Train loss: 0.0010435409283277306
Validation loss: 0.0017941044031787264
Validation Accuracy: 0.9995652173913044
F1-Score: 0.5238095238095238



Epoch: 100%|██████████| 5/5 [18:27<00:00, 221.37s/it]


In [0]:
model.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/bert_state_dict_partial.pt'))

<All keys matched successfully>

In [0]:
checkpoint = torch.load('/content/drive/My Drive/Colab Notebooks/bert_epoch_state_full.pt')
model.load_state_dict(checkpoint['model_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']


In [0]:





def preprocess_test_data(data):
    words = list(set(data["Word"].values))
    words.append("ENDPAD")
    n_words = len(words); n_words
    #print(words[1:10])
    
    
    tags = list(set(data["tag"].values))
    n_tags = len(tags); n_tags
    print(tags[0:n_tags])
    
    
    getter = SentenceGetter(data)
    
    
    sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
    sentences[23]
    
    
    labels = [[s[1] for s in sent] for sent in getter.sentences]
    print(len(labels))
    
    
    
    tags_vals = list(set(data["tag"].values))
    tag2idx = {t: i for i, t in enumerate(tags_vals)}

    print(tag2idx)


    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    print(tokenized_texts)



    input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAXLEN, dtype="long", truncating="post", padding="post")
    input_ids[0]


    tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAXLEN, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")

    len(tags)

  
    attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

    #attention_masks



    tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2019, test_size=0.9)
    tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2019, test_size=0.9)


    tr_inputs = torch.tensor(tr_inputs).to(torch.int64)
    val_inputs = torch.tensor(val_inputs).to(torch.int64)
    tr_tags = torch.tensor(tr_tags).to(torch.int64)
    val_tags = torch.tensor(val_tags).to(torch.int64)
    tr_masks = torch.tensor(tr_masks).to(torch.int64)
    val_masks = torch.tensor(val_masks).to(torch.int64)





    test_data = TensorDataset(val_inputs, val_masks, val_tags)
    test_sampler = SequentialSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=bs)


    return test_dataloader




  


In [0]:
data_test = pd.read_csv("/content/drive/My Drive/NER_Disease/train.csv", encoding="latin1")
data_test = data_test.fillna(method="ffill")
data.tail(10)

data_sample=data_test.sample(n=1000,random_state=1234)
data_sample.tail(10)

test_dataloader= preprocess_test_data(data_sample)


['B-indications', 'O', 'I-indications']
995
{'B-indications': 0, 'O': 1, 'I-indications': 2}
[['significantly'], ['0', '.', '64'], ['and'], ['def', '##ae', '##cation'], ['abnormal', '##ity'], [','], ['.'], ['hybrid', '-', 'cell'], ['at'], ['of'], ['differential'], ['organ', '##ome', '##tal', '##lic'], ['a'], ['field'], ['paint', '##brush'], ['neuroscience'], ['these'], ['fl', '##a', '.'], ['complex'], ['species'], ['be'], ['had'], ['('], ['ve', '##rte', '##brates'], ['lines'], ['p'], ['also'], ['('], [','], ['of'], ['('], ['homosexuality'], ['nucleus'], ['cellular'], ['overall'], ['do'], ['in'], ['.'], ['were'], [','], ['have'], ['to'], ['treatment'], ['and'], ['pl', '##oid', '##y'], ['with'], ['also'], ['acid'], ['group'], ['fertile'], ['may'], ['without'], ['2018'], ['find'], ['der'], ['yet'], ['both'], ['.'], ['at'], ['and'], ['self', '-', 'administration'], ['poly', '##p'], ['up'], ['er', '##ite', '##mat', '##oso'], ['do', '##pa', '##mine', '##rg', '##ic'], ['multiple'], ['item', '

Evaluate The Model

In [0]:
model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

#print(predictions)

Validation loss: 0.0017941044031787264
Validation Accuracy: 0.9995652173913044
Validation F1-Score: 0.5238095238095238


None
